In [1]:
import json
import openai
from bs4 import BeautifulSoup as bs
import requests
import config

In [2]:
# Reading the API key from a file
service_key = config.api_key
openai.api_key = service_key

In [3]:
# PACKAGE NAME
pkg_name = "react"

In [4]:
# Get the URL
URL = 'https://snyk.io/advisor/npm-package/' + pkg_name
URL_dep = URL + '#dependencies'
r =requests.get(URL)
r_dep =requests.get(URL_dep)

In [5]:
# TODO: Parse the correct URL
# with open("myresponse.html") as fp:
#     soup = bs(fp, "html.parser")
soup = bs(r.text, "html.parser")
soup_dep = bs(r_dep.text, "html.parser")

In [6]:
# Parse the HTML for the data
data_set = []
dependencies = ['List of package dependencies']
for tag in soup.find_all('div', class_='number'):
    data_set.append(tag.text)
for tag in soup.find_all('ul', class_='scores'):
    for ele in tag.find_all('li'):
        data_set.append(ele.text)
for tag in soup.find_all('dl', class_='stats'):
    for ele in tag.find_all('div'):
        text = ele.text.replace('\n', '')
        text = " ".join(text.split())
        data_set.append(text)
# Dependencies
for tag in soup_dep.find_all('div', {'id': 'dependencies'}):
    for ele in tag.find_all('a'):
#         print(ele.text)
        dependencies.append(ele.text)

In [7]:
# data_set

In [8]:
# Create list of JSON objects
list_of_jsons = []
for line in data_set:
    json_obj = {"text": line}
    list_of_jsons.append(json_obj)
dep = ", ".join(dependencies)
dep_obj = [{'text' : dep, "metadata": "list of package dependencies"}]

In [9]:
# Create file with data
with open('data.jsonl', 'w') as json_file:
    for ele in list_of_jsons:
        json.dump(ele, json_file)
        json_file.write('\n')
    for ele in dep_obj:
        json.dump(ele, json_file)
        json_file.write('\n')

In [10]:
# Send file to openAI
file = openai.File.create(file=open("data.jsonl"), purpose='answers')
file_id = file['id']
file_id

Upload progress: 100%|██████████| 1.20k/1.20k [00:00<00:00, 2.56kit/s]


'file-Kq2B97nrlC1X12sWb3Oxjhoy'

In [32]:
# QUESTION:
q = 'How many dependencies?'

In [33]:
# Get an answer
answer = openai.Answer.create(
    search_model="ada", 
    model="curie", 
    question=q, 
    file=file_id, 
    examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
    examples=[["What is human life expectancy in the United States?", "78 years."]], 
    max_rerank=10,
    max_tokens=5,
    stop=["\n", "<|endoftext|>"]
)

In [34]:
ans = "".join(answer['answers'])
print(f"Q: {q}\nA: {ans}")

Q: How many dependencies?
A: 2.
